In [1]:
!pip install geopandas osmnx requests


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\jiang\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
import requests

# 获取纽约市所有公园的数据
response = requests.get("https://data.cityofnewyork.us/resource/enfh-gkve.json")
nyc_parks = response.json()

# 检查数据结构
print(nyc_parks[0])  # 打印第一个元素，检查字段名称


{'acquisitiondate': '1937-06-01 00:00:00.0000000', 'acres': '249.389', 'borough': 'Q', 'class': 'PARK', 'communityboard': '401403404406', 'councildistrict': '1921222429', 'department': 'Q-03', 'eapply': 'Grand Central Parkway Ext', 'gisobjid': '100000441', 'gispropnum': 'Q084A', 'globalid': '14567485-5E02-4AD0-B8B6-516D78FC8636', 'jurisdiction': 'DPR/CDOT/SDOT', 'location': 'Astoria Blvd. and 48 St. to Union Tp., Park Drive East.', 'mapped': 'True', 'name311': 'Grand Central Parkway Extension', 'nys_assembly': '273536', 'nys_senate': '13141516', 'objectid': '15508', 'omppropid': 'Q084A', 'parentid': 'Q-03', 'permit': True, 'permitdistrict': 'Q-03', 'permitparent': 'Q-03', 'pip_ratable': False, 'precinct': '114', 'retired': False, 'signname': 'Grand Central Parkway Extension', 'subcategory': 'EXWY', 'typecategory': 'Parkway', 'url': 'http://www.nycgovparks.org/parks/Q084A/', 'us_congress': '614', 'waterfront': True, 'zipcode': '11103113671136811369113701137111375', 'multipolygon': {'typ

In [12]:
{
    "the_geom": {
        "coordinates": [-74.017028, 40.703277],
        "type": "Point"
    },
    "name": "Battery Park"
}


{'the_geom': {'coordinates': [-74.017028, 40.703277], 'type': 'Point'},
 'name': 'Battery Park'}

In [15]:
import requests
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
import matplotlib.pyplot as plt

# 获取纽约市所有公园的数据
response = requests.get("https://data.cityofnewyork.us/resource/enfh-gkve.json")
nyc_parks = response.json()

# 提取经纬度并转换为 GeoDataFrame
nyc_parks_df = pd.DataFrame(
    {
        "park_id": [x.get("park_id", "na") for x in nyc_parks],
        "park_name": [x.get("park_name", "na") for x in nyc_parks],
        "longitude": [x.get("geocoding", {}).get("coordinates", [None, None])[0] for x in nyc_parks],
        "latitude": [x.get("geocoding", {}).get("coordinates", [None, None])[1] for x in nyc_parks]
    }
)

# 去除无效数据
nyc_parks_df.dropna(subset=["longitude", "latitude"], inplace=True)

# 转换为 GeoDataFrame
nyc_parks = gpd.GeoDataFrame(
    nyc_parks_df,
    geometry=gpd.points_from_xy(nyc_parks_df.longitude, nyc_parks_df.latitude),
    crs="EPSG:4326"
)

# 填充缺失值，并去重
nyc_parks.fillna("na", inplace=True)
nyc_parks.drop_duplicates(subset="park_id", inplace=True)

# 定义我的公寓位置
my_apartment = Point(-73.96044, 40.81066)  # 434 W 120th St. New York

# 获取网络数据
bbox = nyc_parks.total_bounds
north, south, east, west = bbox[3], bbox[1], bbox[2], bbox[0]
print("Corrected Bounding Box:", (north, south, east, west))

network = ox.graph_from_bbox(
    north, south, east, west, network_type="walk"
)

# 获取节点和边数据
net_nodes, net_edges = ox.graph_to_gdfs(network)

# 找到最近的网络节点
orig = ox.distance.nearest_nodes(network, my_apartment.x, my_apartment.y)

# 定义计算距离的函数
def distance_from_apartment(row, origin):
    dest = ox.distance.nearest_nodes(network, row.geometry.x, row.geometry.y)
    route = ox.shortest_path(network, origin, dest, weight="length")
    edge_lengths = ox.routing.route_to_gdf(network, route)["length"]
    route_geom = ox.routing.route_to_gdf(network, route).geometry.unary_union
    return {"distance": sum(edge_lengths) / 1609.34, "route_geom": route_geom}

# 计算每个公园到公寓的距离
nyc_parks["distance_from_apartment"] = nyc_parks.apply(
    distance_from_apartment, origin=orig, axis=1
)

nyc_parks = pd.concat(
    [
        nyc_parks.drop(["distance_from_apartment"], axis=1),
        nyc_parks["distance_from_apartment"].apply(pd.Series),
    ],
    axis=1,
)

# 创建一个 GeoDataFrame 来存储路线
parks_routes = gpd.GeoDataFrame(nyc_parks, geometry="route_geom", crs="EPSG:4326")

# 绘制地图
ax = gpd.GeoSeries(my_apartment).plot(color="black")
net_edges.plot(ax=ax, color="black", linewidth=0.1)
parks_routes.plot(column="distance", cmap="magma", ax=ax, linewidth=3)
nyc_parks.plot(column="distance", cmap="magma", legend=True, ax=ax).set_axis_off()

for x, y, label in zip(nyc_parks.geometry.x, nyc_parks.geometry.y, nyc_parks.park_name):
    ax.text(x, y, label, fontsize=6)
ax.text(my_apartment.x, my_apartment.y, "My Apartment", fontsize=6)

plt.title("Distance from My Apartment to NYC Parks")
plt.show()

# 输出距离排序的公园列表
print(nyc_parks[["park_name", "distance"]].sort_values("distance"))


Corrected Bounding Box: (nan, nan, nan, nan)


C:\Users\jiang\AppData\Local\Temp\ipykernel_49136\2628913487.py:44: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  network = ox.graph_from_bbox(


GEOSException: IllegalArgumentException: Points of LinearRing do not form a closed linestring